# Sampling from Light Tail Distribution: Time Step

In this notebook we consider a simple and natural case where the potential has non-Lipschitz gradient.
Our goal is to sample from the density 
\begin{equation*}
\mu^*(x) \propto \exp{\big(-\tfrac{|x|^4}{4} \big)} \, ,
\end{equation*}
which is a~stationary distribution of the process 
\begin{equation*}
\mathrm{d} Y_t = -Y_t^3 \, \mathrm{d} t + \sqrt{2} \, \mathrm{d} B_t \,.
\end{equation*}

We provide the test with respect to time step $\tau$.

Load neccessary libraries.

In [2]:
import numpy as np
from tqdm import tqdm
import scipy.stats as sp
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from IPython.display import clear_output

### Define the Parameters of the Run:

In [12]:
# PARAMETERS OF THE RUN
initial_sample = 0*np.ones(1000)
d = len(initial_sample)
time_step_vector = np.array([1e-5, 1e-4, 1e-3, 0.25, 0.5, 0.75, 1])
sampling = 10**5
n_MC = 3
tune_interval = 10**4
number_of_samples = sampling+tune_interval

For the compuation of the reference values of moments $\mathbb{E} |Y|^2$, $\mathbb{E} |Y|^4$, and $\mathbb{E} |Y|^6$, se the Juputer notebook $\mathtt{reference.ipynb}$. In this notebook, we just assign the computed values.

In [5]:
#reference value 
EX2 = 31.61232243
EX4 = 1000.318801
EX6 = 31684.37599

To simplify code latter, we introduce following functions which return relative error (RE) and coefficient of variance (CV). 

In [6]:
# define a potential and its gradient
def potential(x:float) -> float:
    return np.linalg.norm(x)**4/4

def gradient(x:float) -> float:
    return np.linalg.norm(x)**2 * x

def hessian_p(x: float, p:float) -> float:
    return 2 * x * (np.dot(x,p)) + np.linalg.norm(x)**2 * p  # + 1/time_step * p

def comp_moment(samples, power=2):
    norms = np.linalg.norm(samples, axis=1)
    moment = np.mean(norms**power)
    return moment

### Unadjusted Langevin Algorithm (ULA)

In [10]:
ula_time_step_re = np.zeros(len(time_step_vector))
ula_time_step_cv = np.zeros(len(time_step_vector))

with tqdm(total=len(time_step_vector)*n_MC*number_of_samples) as pbar:

    for i_ts in range(0,len(time_step_vector)):

        time_step = time_step_vector[i_ts]
        moment = np.zeros(n_MC)

        for i_MC in range(0, n_MC):

            samples_ula = np.empty((number_of_samples, d))
            samples_ula[0,:] = initial_sample

            for i_sample in range(1, number_of_samples):

                samples_ula_prev = samples_ula[i_sample-1,:]
                #gradient step
                x = samples_ula_prev - time_step * gradient(samples_ula_prev)  
                
                # adding Gaussian
                x = x + sp.norm.rvs(loc=0, scale=np.sqrt(2*time_step), size=d)
                
                # save value
                samples_ula[i_sample,:] = x

                pbar.update(1)

            samples_ula = samples_ula[tune_interval:]

            moment[i_MC] = comp_moment(samples_ula, 2)
            
        ula_time_step_re[i_ts] = np.abs(np.mean(moment)-EX2)/EX2
        ula_time_step_cv[i_ts] = np.std(moment)/np.mean(moment)

 20%|█▉        | 328053/1650000 [00:11<00:42, 30966.79it/s]/var/folders/82/tsl217q50tv6x9csq7yp35c40000gn/T/ipykernel_26888/2151320655.py:6: RuntimeWarning: overflow encountered in multiply
  return np.linalg.norm(x)**2 * x
/var/folders/82/tsl217q50tv6x9csq7yp35c40000gn/T/ipykernel_26888/3705117481.py:20: RuntimeWarning: invalid value encountered in subtract
  x = samples_ula_prev - time_step * gradient(samples_ula_prev)
100%|█████████▉| 1649985/1650000 [00:57<00:00, 28529.42it/s]


### Tamed Unadjusted Langevin Algorithm (TULA)

In [9]:
tula_time_step_re = np.zeros(len(time_step_vector))
tula_time_step_cv = np.zeros(len(time_step_vector))

with tqdm(total=len(time_step_vector)*n_MC*number_of_samples) as pbar:

    for i_ts in range(0,len(time_step_vector)):

        time_step = time_step_vector[i_ts]
        moment = np.zeros(n_MC)

        for i_MC in range(0, n_MC):

            samples_tula = np.empty((number_of_samples, d))
            samples_tula[0,:] = initial_sample

            for i_sample in range(1, number_of_samples):

                samples_tula_prev = samples_tula[i_sample-1,:]
                
                # gradient tamed step
                x = samples_tula_prev - time_step * gradient(samples_tula_prev) / (1+time_step*np.linalg.norm(gradient(samples_tula_prev))) 
                
                # adding Gaussian
                x = x + sp.norm.rvs(loc=0, scale=np.sqrt(2*time_step), size=d)
            
                # save value
                samples_tula[i_sample,:] = x

                pbar.update(1)

            samples_tula = samples_tula[tune_interval:]

            moment[i_MC] = comp_moment(samples_tula, 2)

        moment = np.array(moment)

        tula_time_step_re[i_ts] = np.abs(np.mean(moment)-EX2)/EX2
        tula_time_step_cv[i_ts] = np.std(moment)/np.mean(moment)


100%|█████████▉| 1649985/1650000 [01:03<00:00, 25830.79it/s]


### Inexact Proximal Langevin Algorithm (IPLA)

In [7]:
ipla_time_step_re = np.zeros(len(time_step_vector))
ipla_time_step_cv = np.zeros(len(time_step_vector))

with tqdm(total=len(time_step_vector)*n_MC*number_of_samples) as pbar:

    for i_ts in range(0,len(time_step_vector)):

        time_step = time_step_vector[i_ts]
        moment = np.zeros(n_MC)

        for i_MC in range(0, n_MC):

            samples_ipla = np.empty((number_of_samples, d))
            samples_ipla[0,:] = initial_sample

            for i_sample in range(1, number_of_samples):

                samples_ipla_prev = samples_ipla[i_sample-1,:]

                # inexact proximal step 
                x = minimize(
                    lambda x: potential(x) + 1/(2*time_step) * np.linalg.norm(x - samples_ipla_prev)**2, 
                    jac=lambda x: gradient(x) + 1/time_step * (x - samples_ipla_prev),
                    hessp=lambda x, p: hessian_p(x,p) + 1/time_step * p,
                    x0=samples_ipla_prev, 
                    method="Newton-CG"
                    ).x
            
                # adding Gaussian
                x = x + sp.norm.rvs(loc=0, scale=np.sqrt(2*time_step), size=d)
                
                # save value
                samples_ipla[i_sample,:] = x

                pbar.update(1)

            samples_ipla = samples_ipla[tune_interval:]

            moment[i_MC] = comp_moment(samples_ipla, 2)

        moment = np.array(moment)

        ipla_time_step_re[i_ts] = np.abs(np.mean(moment)-EX2)/EX2
        ipla_time_step_cv[i_ts] = np.std(moment)/np.mean(moment)


100%|█████████▉| 1649985/1650000 [11:20<00:00, 2423.46it/s]


In [1]:
plt.rcParams.update({
    "font.family": "serif",   # specify font family here
    "font.serif": ["Times"],  # specify font here
    "text.usetex": True,
    } 
    )

# # find if (and when) ULA blow up

# is_nan = False  
# for i_ts in range(0, len(time_step_vector)):
#     if np.isnan(ula_time_step_re[i_ts]):
#         is_nan = True
#         break

# print(i_ts)

plt.figure(figsize=(3,2))
plt.plot(time_step_vector, ipla_time_step_re, color="C0", label="IPLA")
plt.plot(time_step_vector, tula_time_step_re, color="C1", label="TULA")
plt.plot(time_step_vector, ula_time_step_re, color="C2", label="ULA")
plt.legend()
plt.xlabel("$\\tau$")
plt.ylabel("RE")
plt.tight_layout()


plt.figure(figsize=(3,2))
plt.plot(time_step_vector, ipla_time_step_cv, color="C0", label="IPLA")
plt.plot(time_step_vector, ula_time_step_cv, color="C1", label="TULA")
plt.plot(time_step_vector, ula_time_step_cv, color="C2", label="ULA")
plt.legend()
plt.xlabel("$\\tau$")
plt.ylabel("CV")
plt.tight_layout()

NameError: name 'plt' is not defined